In [6]:
import numpy as np 
import pandas as pd

In [29]:
class Network(object):
    
    def __init__(self, layer_node): #layer_node is a list contains the number of nodes in each layer
        self.num_layer = len(layer_node)
        self.layer_node = layer_node
        self.input_node = []
        #self.node = [ np.random.randn(num_node,1) for num_node in layer_node ]
        self.weights = [ np.random.randn(next_node,previous_node+1) for next_node, previous_node in zip(layer_node[1:],layer_node[:-1]) ]
        
    def add_input_node(self,input_vector): #must be of size (n,1)
        if len(input_vector) == self.layer_node[0]: 
            self.input_node = np.append([[1]],input_vector, axis=0) # add bias node
        else:
            print "Input vector length invalid: given len {1}, need len {0}".format(self.layer_node[0], len(input_vector))
            
    def forward_propagation(self):
        output_vector = self.input_node
        for weight in self.weights: 
            #print weight.shape
            #print output_vector.shape
            s = np.dot(weight, output_vector)
            output_vector = np.append([[1]], output_transformation(s), axis=0) # add bias node -> size (n+1,1)
        return output_vector[1:] # we omit the bias node 
        
    def backpropagation(self,input_vector, output_vector):
        s = [] # vector before transformation 
        x = [np.append([[1]],input_vector, axis=0)] # vector after transformation
        sensitivity = [ np.zeros([1,i]) for i in self.layer_node ]
        
        #feedforward, save all the node coefs of each layer
        for weight in self.weights:
            s.append(np.dot(weight, x[-1]))
            x.append(np.append([[1]], output_transformation(s[-1]), axis=0))
            
        sensitivity[-1] = 2*(x[-1][1:] - output_vector) # omit the bias node 
        
        #backpropagation, compute sensitivity backward
        for layer_index in xrange(2, self.num_layer+1):
            w = self.weights[-layer_index+1]
            #print w
            #print w.shape
            print sensitivity[-layer_index+1].shape
            
            sensitivity[-layer_index] = (np.dot(w.T,sensitivity[-layer_index+1]))[1:] #excluding the bias componnent which has the index 0
            
        return sensitivity
        
         
            

In [30]:
def output_transformation(z):
    return z # linear regression -> identity function

In [31]:
input_signal = np.array([[1],[2]])
input_signal

array([[1],
       [2]])

In [32]:
net = Network([2,3,1])

In [33]:
net.add_input_node(input_signal)

In [34]:
net.forward_propagation()

array([[ 2.43388697]])

In [35]:
s = net.backpropagation(input_signal,np.array([10]))

[[-0.37232878 -0.74908326  1.20911348  0.51379664]]
(1, 4)
(1, 1)
[[-0.54408268 -0.53440993 -1.54756976]
 [-0.4904078   0.85622078 -0.42948791]
 [-0.00864444 -0.36055841  0.45329494]]
(3, 3)
(3, 1)


In [37]:
s

[array([[-18.92030515],
        [-13.20832069]]), array([[ 11.33529717],
        [-18.29657848],
        [ -7.77488691]]), array([[-15.13222607]])]